In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam

In [3]:
# Generating synthetic data
num_users = 1000
num_products = 500
interaction_data = []

for _ in range(5000):
    user = np.random.randint(0, num_users)
    product = np.random.randint(0, num_products)
    interaction = np.random.uniform(0, 5)  # Simulate interaction scores between 0 and 5
    interaction_data.append([user, product, interaction])

interaction_df = pd.DataFrame(interaction_data, columns=['user_id', 'product_id', 'interaction'])

In [4]:
# Preprocessing the data
encoder = LabelEncoder()
interaction_df['user_id'] = encoder.fit_transform(interaction_df['user_id'])
interaction_df['product_id'] = encoder.fit_transform(interaction_df['product_id'])

scaler = MinMaxScaler()
interaction_df['interaction'] = scaler.fit_transform(interaction_df['interaction'].values.reshape(-1, 1))

train_data, test_data = train_test_split(interaction_df, test_size=0.2, random_state=42)

In [5]:
# Model design
user_input = Input(shape=(1,))
product_input = Input(shape=(1,))
user_embedding = Embedding(input_dim=num_users, output_dim=50)(user_input)
product_embedding = Embedding(input_dim=num_products, output_dim=50)(product_input)
user_flat = Flatten()(user_embedding)
product_flat = Flatten()(product_embedding)
concatenated = Concatenate()([user_flat, product_flat])
fc1 = Dense(128, activation='relu')(concatenated)
dropout1 = Dropout(0.3)(fc1)
fc2 = Dense(64, activation='relu')(dropout1)
dropout2 = Dropout(0.3)(fc2)
output = Dense(1, activation='linear')(dropout2)

model = Model(inputs=[user_input, product_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

In [6]:
# Model training
model.fit([train_data['user_id'], train_data['product_id']], train_data['interaction'],
          batch_size=64, epochs=20, validation_split=0.1)

Epoch 1/20
57/57 [==============================] - 2s 10ms/step - loss: 0.1251 - val_loss: 0.0773
Epoch 2/20
57/57 [==============================] - 0s 4ms/step - loss: 0.0807 - val_loss: 0.0795
Epoch 3/20
57/57 [==============================] - 0s 4ms/step - loss: 0.0722 - val_loss: 0.0873
Epoch 4/20
57/57 [==============================] - 0s 6ms/step - loss: 0.0627 - val_loss: 0.0924
Epoch 5/20
57/57 [==============================] - 0s 4ms/step - loss: 0.0557 - val_loss: 0.0960
Epoch 6/20
57/57 [==============================] - 0s 3ms/step - loss: 0.0499 - val_loss: 0.1015
Epoch 7/20
57/57 [==============================] - 0s 4ms/step - loss: 0.0424 - val_loss: 0.1008
Epoch 8/20
57/57 [==============================] - 0s 4ms/step - loss: 0.0363 - val_loss: 0.1048
Epoch 9/20
57/57 [==============================] - 0s 5ms/step - loss: 0.0288 - val_loss: 0.1055
Epoch 10/20
57/57 [==============================] - 0s 4ms/step - loss: 0.0242 - val_loss: 0.1085
Epoch 11/20
57/57 

In [7]:
# Generating personalized rankings for a specific user
user_id = encoder.transform([10])  # Can be replace with the desired user ID
product_ids = np.arange(num_products)
product_ids_encoded = encoder.transform(product_ids)
ranking_scores = model.predict([np.array([user_id] * num_products), product_ids_encoded]).flatten()

16/16 [==============================] - 0s 2ms/step


In [8]:
# Sort product IDs based on ranking scores to get top 10 recommended products
top_product_indices = np.argsort(ranking_scores)[::-1][:10]

In [9]:
# Displaying top 10 recommended products
print("Top 10 Recommended Products for USER ID:",user_id[0])
for rank, product_idx in enumerate(top_product_indices, start=1):
    original_product_id = encoder.inverse_transform([product_idx])[0]
    print(f"Rank {rank}: Product ID:{original_product_id} (Ranking Score: {ranking_scores[product_idx]:.4f})")

Top 10 Recommended Products for USER ID: 10
Rank 1: Product ID:36 (Ranking Score: 0.9441)
Rank 2: Product ID:421 (Ranking Score: 0.9412)
Rank 3: Product ID:353 (Ranking Score: 0.8885)
Rank 4: Product ID:254 (Ranking Score: 0.8841)
Rank 5: Product ID:82 (Ranking Score: 0.8615)
Rank 6: Product ID:349 (Ranking Score: 0.8326)
Rank 7: Product ID:226 (Ranking Score: 0.8240)
Rank 8: Product ID:149 (Ranking Score: 0.8216)
Rank 9: Product ID:108 (Ranking Score: 0.8184)
Rank 10: Product ID:475 (Ranking Score: 0.8153)
